In [1]:
# pip install earthengine-api --upgrade

# Authorization code="4/1AfJohXnfQF5LmLzwPElBhPeee8d1X9folqXwP91lrbPnQtaBuLZHnF24IlE"

In [2]:
# pip install geemap

In [3]:
import ee
import geemap
# ee.Authenticate()
ee.Initialize()
# print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())

In [4]:


# Define date ranges
before_start = '2018-08-07'
before_end = '2018-08-16'
after_start = '2018-08-16'
after_end = '2018-08-23'

# Other parameters
polarization = "VH"
pass_direction = "DESCENDING"
difference_threshold = 1.25

# Load Sentinel-1 GRD data and filter
collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
              .filter(ee.Filter.eq('instrumentMode', 'IW'))
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization))
              .filter(ee.Filter.eq('orbitProperties_pass', pass_direction))
              .filter(ee.Filter.eq('resolution_meters', 10)))

# Load the shapefile as a FeatureCollection
geometry = ee.FeatureCollection("users/amanchaudhary/periyar_basin")


# Select images by predefined dates
before_collection = collection.filterDate(before_start, before_end)
after_collection = collection.filterDate(after_start, after_end)

# Print selected tiles to the console
before_count = before_collection.size().getInfo()
print(f'Tiles selected: Before Flood ({before_count})', before_start, before_end)
after_count = after_collection.size().getInfo()
print(f'Tiles selected: After Flood ({after_count})', after_start, after_end)

# Create a mosaic of selected tiles and clip to the study area
before = before_collection.mosaic().clip(geometry)
after = after_collection.mosaic().clip(geometry)

# Apply radar speckle reduction by smoothing
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters')
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters')

# Water Detection
waterThreshold = -18
afterWaterBinary = after_filtered.lt(waterThreshold).selfMask()

# Flood Extent Calculation
difference = after_filtered.divide(before_filtered)
threshold = difference_threshold
difference_binary = difference.gt(threshold)

# Refine flood result using additional datasets
swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality')
swater_mask = swater.gte(10).updateMask(swater.gte(10))
flooded_mask = difference_binary.where(swater_mask, 0)
connections = flooded_mask.connectedPixelCount()
flooded = flooded_mask.updateMask(connections.gte(8))
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM')
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
flooded = flooded.updateMask(slope.lt(5))

# Calculate flood extent area
flood_pixelarea = flooded.select(polarization).multiply(ee.Image.pixelArea())
flood_stats = flood_pixelarea.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=geometry,
    scale=10,
    bestEffort=True
)

# Display Products using geemap
# Map = geemap.Map()
# Map.centerObject(geometry, 8)
# Map.addLayer(before_filtered, {'min': -25, 'max': 0}, 'Before Flood', 0)
# Map.addLayer(after_filtered, {'min': -25, 'max': 0}, 'After Flood', 1)
# Map.addLayer(afterWaterBinary, {'palette': 'blue'}, 'Water After Flood')
# Map.addLayer(difference, {'min': 0, 'max': 2}, 'Difference Layer', 0)
# Map.addLayer(flooded, {'palette': "ff0051"}, 'Flooded areas')
# Map

# Export the 'afterWaterBinary' image as a raster file
export_params = {
    'image': afterWaterBinary,
    'description': 'afterWaterBinary',
    'scale': 10,
    'region': geometry.geometry().bounds(),
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e10
}


task = ee.batch.Export.image.toDrive(**export_params)
task.start()


Tiles selected: Before Flood (3464) 2018-08-07 2018-08-16
Tiles selected: After Flood (2863) 2018-08-16 2018-08-23


In [7]:
# Check the status of the export task
status = task.status()
print('Export task status:', status)

# If the task has completed, print the link to the exported file
if status['state'] == 'COMPLETED':
    print('Export completed. Download link:', status['id'])


Export task status: {'state': 'RUNNING', 'description': 'afterWaterBinary', 'creation_timestamp_ms': 1695055732235, 'update_timestamp_ms': 1695055746212, 'start_timestamp_ms': 1695055746178, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': '3CVOXLU3I422ESWRB4FIOQRT', 'name': 'projects/earthengine-legacy/operations/3CVOXLU3I422ESWRB4FIOQRT'}
